In [29]:
import os
import pandas as pd
import numpy as np
#from matplotlib import pyplot as plt
from scipy import ndimage as ndi
from skimage import (exposure, feature, filters, io, measure,
                      morphology, restoration, segmentation, transform,
                      util)
import napari
from sklearn.cluster import HDBSCAN
from sklearn.mixture import BayesianGaussianMixture
import shutil
import tifffile

In [50]:
fp_starter = os.path.abspath('../') + '/'

#load 1 raw bug
#works for sure on mix_02_006.tif
chosen_image = "mix_02_006.tif"
bug = io.imread(fp_starter + 'BugNIST_DATA/validation/' + chosen_image)

print("shape: {}".format(bug.shape))
print("dtype: {}".format(bug.dtype))
print("range: ({}, {})".format(np.min(bug), np.max(bug)))

shape: (128, 92, 92)
dtype: uint8
range: (0, 249)


In [51]:
#DataFrame with all given validation centerpoints 
valid_fp = fp_starter + 'BugNIST_DATA/validation/'

valid_y = pd.read_csv(valid_fp+"/validation.csv")
valid_y['bugs_present']=valid_y['centerpoints'].str.split(";").str[0::4]
valid_y['n_bugs']=valid_y['centerpoints'].str.split(";").str[0::4].str.len()
valid_y['z'] = valid_y['centerpoints'].str.split(";").str[1::4]
valid_y['y'] = valid_y['centerpoints'].str.split(";").str[2::4]
valid_y['x'] = valid_y['centerpoints'].str.split(";").str[3::4]
valid_y['centerpoint_coords'] = valid_y.apply(lambda x: [[x['x'][i], x['y'][i], x['z'][i]] for i in range(len(x['x']))], axis=1)

#grab centerpoints for chosen image
validation_coordinates = [[float(y) for y in x] for x in valid_y[valid_y['filename']==chosen_image]['centerpoint_coords'].to_list()[0]]

In [52]:
valid_y

,filename,centerpoints,bugs_present,n_bugs,z,y,x,centerpoint_coords
0,mix_02_006.tif,bl;48.61;39.14;96.19;bl;77.71;53.14;57.17;pp;5...,"[bl, bl, pp, pp, pp, pp, pp, bc, bc, bc]",10,"[48.61, 77.71, 58.71, 48.42, 19.84, 22.67, 29....","[39.14, 53.14, 73.71, 45.33, 62.44, 30.00, 42....","[96.19, 57.17, 59.15, 51.75, 40.23, 53.13, 60....","[[96.19, 39.14, 48.61], [57.17, 53.14, 77.71],..."
1,mix_02_011.tif,ml;77.86;46.26;57.79;sl;41.14;61.93;32.37;sl;5...,"[ml, sl, sl, cf, cf, cf]",6,"[77.86, 41.14, 56.09, 34.21, 52.70, 46.03]","[46.26, 61.93, 48.52, 70.82, 61.51, 15.26]","[57.79, 32.37, 56.20, 53.24, 44.71, 35.94]","[[57.79, 46.26, 77.86], [32.37, 61.93, 41.14],..."
2,mix_03_001.tif,ml;54.20;34.58;81.43;ml;59.60;41.94;60.72;ml;4...,"[ml, ml, ml, ml, ml, wo, wo, wo, wo, wo]",10,"[54.20, 59.60, 40.53, 57.57, 26.76, 20.74, 41....","[34.58, 41.94, 41.62, 58.60, 65.45, 47.13, 24....","[81.43, 60.72, 71.55, 39.59, 55.51, 63.01, 44....","[[81.43, 34.58, 54.20], [60.72, 41.94, 59.60],..."
3,mix_03_003.tif,wo;44.37;27.71;85.18;wo;30.04;28.46;58.76;wo;4...,"[wo, wo, wo, wo, ml, ml, ml, bl, bl, bl, bl, b...",13,"[44.37, 30.04, 46.50, 23.91, 57.88, 55.18, 33....","[27.71, 28.46, 55.14, 47.43, 26.24, 56.66, 44....","[85.18, 58.76, 96.45, 102.13, 60.98, 53.65, 56...","[[85.18, 27.71, 44.37], [58.76, 28.46, 30.04],..."
4,mix_03_004.tif,pp;74.65;44.18;52.53;pp;30.69;25.53;54.51;pp;1...,"[pp, pp, pp, pp, pp, pp, pp, bc, bc, cf, cf, c...",15,"[74.65, 30.69, 16.61, 41.75, 55.04, 44.65, 28....","[44.18, 25.53, 48.47, 72.43, 61.95, 47.29, 50....","[52.53, 54.51, 62.38, 53.18, 48.09, 39.80, 52....","[[52.53, 44.18, 74.65], [54.51, 25.53, 30.69],..."
...,...,...,...,...,...,...,...,...
73,mix_28_008.tif,ml;73.50;47.97;60.62;ml;30.65;24.54;75.91;ml;1...,"[ml, ml, ml, ml, bc, bc, bc, bc, bc]",9,"[73.50, 30.65, 17.95, 38.39, 53.40, 26.39, 60....","[47.97, 24.54, 60.56, 43.09, 18.45, 39.16, 64....","[60.62, 75.91, 87.30, 45.11, 78.67, 30.37, 16....","[[60.62, 47.97, 73.50], [75.91, 24.54, 30.65],..."
74,mix_28_011.tif,cf;67.69;69.75;38.29;cf;46.78;25.38;42.34;ma;5...,"[cf, cf, ma, ma, bc, bc, bc, bc]",8,"[67.69, 46.78, 51.28, 36.34, 25.87, 42.58, 20....","[69.75, 25.38, 66.45, 25.32, 41.03, 23.14, 46....","[38.29, 42.34, 39.66, 62.78, 70.77, 74.99, 54....","[[38.29, 69.75, 67.69], [42.34, 25.38, 46.78],..."
75,mix_29_005.tif,bf;72.06;58.57;38.59;bf;48.71;29.59;47.57;bf;3...,"[bf, bf, bf, ma, ma, bp, bp, bp, bp]",9,"[72.06, 48.71, 32.49, 65.23, 48.57, 50.01, 24....","[58.57, 29.59, 37.20, 45.02, 22.79, 43.96, 59....","[38.59, 47.57, 57.90, 72.38, 73.63, 15.60, 14....","[[38.59, 58.57, 72.06], [47.57, 29.59, 48.71],..."
76,mix_29_009.tif,ml;42.26;26.99;48.13;ml;58.60;25.86;42.10;ml;5...,"[ml, ml, ml, ml, ml, sl, sl, sl, sl, bl, bl, b...",17,"[42.26, 58.60, 50.53, 36.79, 59.78, 49.66, 30....","[26.99, 25.86, 42.58, 50.87, 59.27, 76.17, 69....","[48.13, 42.10, 47.10, 41.22, 46.91, 60.35, 67....","[[48.13, 26.99, 42.26], [42.10, 25.86, 58.60],..."


In [53]:
#view bug and centerpoints in napari

viewer = napari.view_image(
    bug,
    #contrast_limits=[0, 1],
    #scale=spacing, [TODO] look up size of imaging to see if it improves bug insides
    ndisplay=3,
)

viewer.add_points(
    [[float(y) for y in x] for x in valid_y[valid_y['filename']=='mix_02_006.tif']['centerpoint_coords'][0]],
    ndim=3,
    name='validation centerpoints',
    #scale=spacing,
    size=4,
    n_dimensional=True, 
    edge_color='red'
)

viewer.add_labels(
    bug
)


<Labels layer 'bug [1]' at 0x7fdaa000cb80>

In [54]:
#Edge Detection
edges = filters.scharr(bug) #[TODO] look into other filters
## scharr seems to be best filter
#edges = filters.sobel(bug)


viewer.add_image(
    edges,
    #scale=spacing,
    blending='additive',
    colormap='magenta',
)


<Image layer 'edges' at 0x7fdb250ea2b0>

In [55]:
#Thresholding
#Separate bugs from the background

## 3 was way too much smoothing, 1 too little
denoised = ndi.median_filter(bug, size=2) #smoothing technique
#thresholded = denoised > filters.threshold_li(denoised) #[TODO] try different thresholding techniques
thresholded = denoised > filters.threshold_otsu(denoised)


viewer.add_image(
    thresholded,
    #scale=spacing,
    opacity=0.3,
)


<Image layer 'thresholded' at 0x7fdaf03295e0>

In [56]:
inds = np.where(~thresholded)
thresholded[inds] = 0.
mask = ndi.binary_erosion(thresholded, morphology.ball(1), iterations=1)
viewer.add_image(
    mask,
    name='cleaned',
    #scale=spacing,
    opacity=0.3,
);


In [57]:
#Morphology
#Grab the areas surrounding each bug and smooth them
width = 20

remove_holes = morphology.remove_small_holes(
    thresholded, 
    width
)
#[TODO] this may end up removing some bugs, need to test
remove_objects = morphology.remove_small_objects(
    remove_holes, 
    width ** 2,
    connectivity=2
)


viewer.add_image(
    remove_objects,
    name='cleaned',
    #scale=spacing,
    opacity=0.3,
);


In [58]:
#labels connected regions -> good for andrews masks
"""
labels = measure.label(remove_objects)

viewer.add_labels(
    labels,
    #scale=spacing,
    opacity=0.5,
)

viewer.layers['cleaned'].visible = False
"""

"\nlabels = measure.label(remove_objects)\n\nviewer.add_labels(\n    labels,\n    #scale=spacing,\n    opacity=0.5,\n)\n\nviewer.layers['cleaned'].visible = False\n"

In [59]:
#Identify insides of bugs
#find the points farthest from the edges

#euclidian distance from outside to inside to find points furthest inside - think heatmap
transformed = ndi.distance_transform_edt(remove_objects)#, sampling=spacing)

#max grabbing techniques
maxima = morphology.local_maxima(transformed, footprint=morphology.ball(1)) # grabs lots more max's
#maxima = morphology.h_maxima(transformed, 1) #anything higher than 1 missed small bugs

#grab all chosen max
maxima_coords = np.transpose(np.nonzero(maxima))
#or grab all inner points
transformed_coords = np.transpose(np.nonzero(transformed))

"""
viewer.add_points(
    np.transpose(np.nonzero(transformed)),
    name='all insides',
    #scale=spacing,
    size=1,
    n_dimensional=True,  # points have 3D "extent"
)
"""

'\nviewer.add_points(\n    np.transpose(np.nonzero(transformed)),\n    name=\'all insides\',\n    #scale=spacing,\n    size=1,\n    n_dimensional=True,  # points have 3D "extent"\n)\n'

In [60]:
'canberra', 'euclidean', 'mahalanobis', 'manhattan', 'chebyshev', 'rogerstanimoto', 
'infinity', 'jaccard', 'sokalmichener', 'braycurtis', 'pyfunc', 'l2', 'russellrao', 
'cityblock', 'haversine', 'minkowski', 'sokalsneath', 'hamming', 'precomputed', 'dice', 
'seuclidean', 'l1', 'p'

"""2 2 15 manhattan"""


'2 2 15 manhattan'

In [61]:
#Higherarchical Clustering 
#Cluster the inside points and average to find points to intialize watershed

mincs = 2
minsa = 2
csele = 10
metri = 'manhattan'

model = HDBSCAN(
    min_cluster_size=mincs,             #min number of points in a cluster
    min_samples=minsa,                  #amount of points that can be considered start of cluster
    cluster_selection_epsilon=csele,   #clusters closer than this get merged
    store_centers="centroid",
    cluster_selection_method="leaf", ##not sure why this better
    metric=metri
)
model.fit(
    np.transpose(np.nonzero(maxima))    
)
centers = model.centroids_.astype(int)
"""
center_of_cluster = []
for clust in np.unique(clusters):
    center_of_cluster += [np.mean(np.transpose(np.nonzero(maxima))[clusters==clust], axis=0)]
center_of_cluster
"""

viewer.add_points(
    centers,
    name='HDBSCAN ' + str(mincs) + " " + str(minsa) + " " + str(csele) + " " + metri,
    #scale=spacing,
    size=4,
    n_dimensional=True,  # points have 3D "extent"
    edge_color = "yellow",
    blending="translucent_no_depth"
)



<Points layer 'HDBSCAN 2 2 10 manhattan' at 0x7fdb251572e0>

In [62]:
(valid_y["n_bugs"].max(),
valid_y[valid_y["filename"]==chosen_image]["n_bugs"])

(40,
 0    10
 Name: n_bugs, dtype: int64)

In [45]:
from vispy.color.colormap import HSL

In [46]:
#Bayesian Gaussian Mixture
#seems better for getting entire bug instead of splitting, but still connects bugs together
#doesnt work with all possible bug points, max better

#aproximate gaussians for an unknown number of bugs
#https://scikit-learn.org/stable/modules/generated/sklearn.mixture.BayesianGaussianMixture.html#sklearn.mixture.BayesianGaussianMixture
#https://scikit-learn.org/stable/auto_examples/mixture/plot_concentration_prior.html#sphx-glr-auto-examples-mixture-plot-concentration-prior-py

mixtures = BayesianGaussianMixture(
    n_components=min(len(maxima_coords), 50),
    covariance_type='tied',
    tol=1e-3,
    reg_covar=0,
    #max_iter=1500,
    init_params="random",  #get random points instead of kmeans (wont cluster well)
    weight_concentration_prior_type="dirichlet_process",
    weight_concentration_prior=10000,
    mean_precision_prior=0.01,
).fit_predict(maxima_coords)

center_of_mix = []
for mix in np.unique(mixtures):
    center_of_mix += [np.mean(maxima_coords[mixtures==mix], axis=0)]
center_of_mix

viewer.add_points(
    center_of_mix,
    name='gaussian centers',
    #scale=spacing,
    size=4,
    n_dimensional=True,  # points have 3D "extent"
    edge_color = "yellow",
    blending="translucent_no_depth"
)

"""
classified_points = np.zeros(bug.shape)
for i in range(len(mixtures)):
    classified_points[maxima_coords[i][0]][maxima_coords[i][1]][maxima_coords[i][2]] = mixtures[i]


classified_points = []
for curr_mix in np.unique(mixtures):
    classified_points += np.transpose(np.nonzero(maxima))[mixtures==curr_mix]
classified_points
"""
features = {
    'mixture_colors': mixtures/50,
}
colored_points = viewer.add_points(
    maxima_coords,
    name='Gaussian outputs',
    features=features,
    #scale=spacing,
    size=4,
    n_dimensional=True,  # points have 3D "extent"
    face_color = 'mixture_colors',
    face_colormap = HSL(ncolors=50),
    blending="translucent_no_depth"
)

#colored_points.edge_color_mode = 'colormap'
#colored_points.edge_colormap = HSL(ncolors=50)
#colored_points.edge_color = mixtures.astype(int)

mixtures, len(np.unique(mixtures))
np.unique(classified_points)

/Users/andrewcheng/opt/anaconda3/lib/python3.9/site-packages/sklearn/mixture/_base.py:268: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


NameError: name 'classified_points' is not defined

In [17]:
#clusterting attempt:
#distance matrix from key points
"""
from scipy.cluster.hierarchy import average, fcluster, fclusterdata

#create condensed distance matrix
y = pdist(np.transpose(np.nonzero(maxima)))
#perform clustering using average linkage
Z = average(y)
#data, threshold, criterion
fclusterdata(Z, 1, criterion='inconsistent')
"""

"\nfrom scipy.cluster.hierarchy import average, fcluster, fclusterdata\n\n#create condensed distance matrix\ny = pdist(np.transpose(np.nonzero(maxima)))\n#perform clustering using average linkage\nZ = average(y)\n#data, threshold, criterion\nfclusterdata(Z, 1, criterion='inconsistent')\n"

In [18]:
#Clustering attempt:
#find local maxima of distance from edges
"""
from scipy.spatial.distance import pdist

transformed = ndi.distance_transform_edt(remove_objects)#, sampling=spacing)

maxima = feature.corner_peaks(transformed, min_distance=int(0.2*width), p_norm=1)
viewer.add_points(
    maxima,
    name='local max points',
    #scale=spacing,
    size=4,
    n_dimensional=True,  # points have 3D "extent"
    edge_color = "blue"
)

feature.peak_local_max(transformed, min_distance=width//2)
"""

'\nfrom scipy.spatial.distance import pdist\n\ntransformed = ndi.distance_transform_edt(remove_objects)#, sampling=spacing)\n\nmaxima = feature.corner_peaks(transformed, min_distance=int(0.2*width), p_norm=1)\nviewer.add_points(\n    maxima,\n    name=\'local max points\',\n    #scale=spacing,\n    size=4,\n    n_dimensional=True,  # points have 3D "extent"\n    edge_color = "blue"\n)\n\nfeature.peak_local_max(transformed, min_distance=width//2)\n'

In [19]:
#Watershed
#create a matrix like image with zeroes except for centroids
markers = np.zeros(bug.shape, dtype=np.uint32)
marker_indices = tuple(np.round(centers).astype(int).T)
markers[marker_indices] = np.arange(len(centers)) + 1
#grow the corresponding centroids
markers_big = morphology.dilation(markers, morphology.ball(5))

#Segment using Watershed from clustered centers
segmented = segmentation.watershed(
    edges,
    markers_big,
    compactness=0.1, ##grabs more distinct bugs from wshed but cuts bugs
    mask=remove_objects,
)
"""
walk_segmented = segmentation.random_walker(
    remove_objects, markers_big
)
"""
"""
viewer = napari.view_image(
    remove_objects,
    #contrast_limits=[0, 1],
    #scale=spacing,
    ndisplay=3,
)
"""
viewer.add_labels(
    segmented,
    #scale=spacing,
)


<Labels layer 'segmented' at 0x7fdac03e8d30>

In [20]:
np.unique(segmented)

array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int32)

In [21]:
np.arange(len(centers)) + 1

array([1, 2, 3, 4, 5, 6, 7, 8])

In [22]:
#Isolate single bugs & recalculate centers

watershed_center_list = []
single_bug_list = []
for point in np.unique(segmented):
    #print(point)
    ## add back in cylinder noise
    single_bug = np.zeros(bug.shape, dtype=int)
    bug_noise = np.random.randint(1, 4, bug.shape)
    single_bug[bug!=0] = bug_noise[bug!=0]
    single_bug[segmented==point] = bug[segmented==point]

    single_bug_list += [single_bug]
    
    watershed_center = np.mean(np.where(single_bug!=0), axis=1).astype(int)
    watershed_center_list += [watershed_center]


In [23]:
#Sanity check on new centers - roughly the same

viewer = napari.view_image(
    edges,
    #contrast_limits=[0, 1],
    #scale=spacing,
    ndisplay=3,
    opacity=0.2
)

viewer.add_labels(
    segmented,
    #scale=spacing,
    opacity=0.2
)
"""
for i in range(len(single_bug_list)):
    viewer.add_image(
        single_bug_list[i],
        name="bug "+str(i),
    )
"""
viewer.add_points(
    watershed_center_list,
    name='watershed centers',
    #scale=spacing,
    size=4,
    n_dimensional=True,  # points have 3D "extent"
    edge_color = "green",
    blending="translucent_no_depth"
)

viewer.add_points(
    centers,
    name='HDBSCAN ' + str(mincs) + " " + str(minsa) + " " + str(csele) + " " + metri,
    #scale=spacing,
    size=4,
    n_dimensional=True,  # points have 3D "extent"
    edge_color = "yellow",
    blending="translucent_no_depth"
)

viewer.add_points(
    validation_coordinates, #take coordinates from input
    ndim=3,
    name='validation centers',
    #scale=spacing,
    size=4,
    n_dimensional=True, 
    edge_color='red',
    blending="translucent_no_depth"
)

<Points layer 'validation centers' at 0x7fdb22d29640>

In [26]:
#write bug segements to files
if not os.path.exists(fp_starter + 'temp/'):
    os.makedirs(fp_starter + 'temp/')
if os.path.exists(fp_starter + 'temp/'+ chosen_image.split('.')[0]):
    shutil.rmtree(fp_starter + 'temp/'+ chosen_image.split('.')[0])
os.makedirs(fp_starter + 'temp/'+ chosen_image.split('.')[0])
for i in range(len(single_bug_list)):
    out_path = fp_starter + 'temp/'+ chosen_image.split('.')[0] + '/bug_' + str(i) +".tif"
    #os.makedirs(fp_starter + 'temp/'+ chosen_image.split('.')[0])
    tifffile.imsave(out_path, single_bug_list[i])

In [27]:
bug = io.imread('/Users/johndriscoll/Debuggers/temp/mix_03_003/bug_10.tif')
viewer = napari.view_image(
    bug,
    #contrast_limits=[0, 1],
    #scale=spacing,
    ndisplay=3,
    opacity=0.6
)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/johndriscoll/Debuggers/temp/mix_03_003/bug_10.tif'

In [28]:
bug_noise = np.random.randint(1, 4, bug.shape)
bug_zeros = np.zeros(bug.shape, dtype=int)
bug_zeros[bug!=0] = bug_noise[bug!=0]
bug_zeros[segmented==point] = bug[segmented==point]

viewer3 = napari.view_image(
    bug_zeros,
    ndisplay=3
)
viewer3.add_labels(
    bug_zeros
)


<Labels layer 'bug_zeros [1]' at 0x7fdb14b7e250>